# La librería Pandas

Pandas es una librería de python que proporciona unas estructuras de datos fáciles de usar y de alto rendimiento, con objeto de realizar tareas de análisis de datos.

Los objetos principales son los `dataframes`, unas estructuras de datos bidimensionales en forma de tabla (de hecho, podremos tratar los datos y devolverlos a otros formatos con los que querramos trabajar, como veremos).

## Creación de dataframes

Podemos crear un dataframe a partir de varias Series: cada una actuará a modo de registro en nuestra tabla. 

Como ejemplo, crearemos una tabla con tres registros y cuyas columnas sean `Cost`, `Item Purchased` y `Name`, simulando una serie de clientes que compran productos en dos tiendas diferentes de objetos para mascotas.

In [ ]:
import pandas as pd # Así importamos la librería pandas (se suele usar el alias "pd")

# Definimos cada operación con un diccionario que nos da la información relevante
purchase_1 = pd.Series({'Name': 'Chris',
                        'Item Purchased': 'Dog Food',
                        'Cost': 22.50})
purchase_2 = pd.Series({'Name': 'Kevyn',
                        'Item Purchased': 'Kitty Litter',
                        'Cost': 2.50})
purchase_3 = pd.Series({'Name': 'Vinod',
                        'Item Purchased': 'Bird Seed',
                        'Cost': 5.00})

# Combinamos las tres líneas anteriores en un dataframe
df = pd.DataFrame([purchase_1, purchase_2, purchase_3], index=['Store 1', 'Store 1', 'Store 2'])

df

Observamos que basta proporcionarle una serie por cada entrada en nuestra tabla. Además, los campos de la Serie deben ser los mismos a lo largo de cada registro, con el fin de que se pueda formar correctamente la tabla.
El argumento index sirve para introducir identificadores a cada registro. Por defecto será un número entero, pero en este caso hemos utilizado strings.

Sin embargo, iremos viendo que hay muchas formas diferentes de crear y lidiar con los dataframes. Como ejemplo, podemos crear el mismo dataframe de manera simple usando numpy arrays como los que hemos visto ya.

In [ ]:
import numpy as np
df_alternativo = pd.DataFrame( data = np.array([["Chris", "Dog Food", 22.5], 
                                               ["Kevyn", "Kitty Litter", 2.5],
                                               [ "Vinod",  "Bird Seed", 5.0]]),
                              columns = ["Name", "Item Purchased", "Cost"],
                              index = ["Store 1", "Store 1", "Store 3"] )
df_alternativo

Para seleccionar un subconjunto de registros conociendo su índice, podemos utilizar `loc`

In [ ]:
df.loc['Store 1']

Si además queremos especificar una columna en concreto, podemos decírselo separandolo con una coma

In [ ]:
df.loc['Store 1', 'Cost']

A su vez, estos argumentos pueden ser listas (con lo que podemos escoger subconjuntos más complejos de forma más simple)

In [ ]:
df.loc[['Store 1', 'Store 2'], ['Cost','Name']]

Si en lugar de mediante los strings queremos acceder utilizando las posiciones (como enteros), podemos usar `iloc`.
Por ejemplo, el siguiente comando toma las dos primeras filas y todas las columnas (la enumeración funciona igual que la de las listas, que ya hemos visto también).

In [ ]:
df.iloc[:3, :]

<br>
Si queremos borrar algún registro conociendo su índice, basta usar `drop`

In [ ]:
df.drop('Store 1')

Pero... ¡ojo! ¿Qué pasa si imprimimos de nuevo el dataframe después de haber quitado esas entradas..?

In [ ]:
df

¡Obtenemos la versión original! Esto es porque `drop` por defecto crea una copia con el nuevo dataframe actualizado. En lugar de `df2 = df.drop(...)` podemos hacer lo siguiente para que la actualización se realice en el mismo lugar de memoria del objeto `df`

In [ ]:
df.drop('Store 1', inplace=True) # Esto es funcionalmente equivalente a hacer "df = df.drop('Store 1')" 
df

## Carga de dataframes

El principal uso que se le suele dar a los dataframes no es realmente el de trabajar con datos creados desde cero, como hemos hecho. Principalmente, usaremos los dataframes de pandas para trabajar con datos presentes en nuestro disco duro y que queremos analizar (tablas de datos externas). Para ello, pandas cuenta con funciones para leer el contenido de estos archivos, y volcarlos a un dataframe.

Las funciones que se ocupan de esto comienzan por `pd.read_...`. Existen para documentos excel, html, json, txt... Nosotros utilizaremos la más común, `pd.read_csv`, que lee del respectivo archivo .csv (cada fila es un registro de la tabla y los valores están separados por comas, de ahí su nombre - "comma separated values").

A continuación cargaremos datos `tips.csv` en un `dataframe` de `pandas`. Esta base de datos, contiene información acerca de las cuentas de un restaurante. En concreto contiene las siguientes variables:

* `total_bill`: Valor de la cuenta.
* `tip`: Valor de la propina.
* `sex`: Género del pagador (Femal/Male).
* `smoker`: Variable categórica que indica si el pagador es o no fumador (Yes/No).
* `day`: Día de la semana.
* `time`: Variable categórica que indica si la cuenta corresponde a una comida o una cena (Dinner/Lunch).
* `size`: Número de comensales en la mesa.

Para cargar un archivo hay que especificar la ruta hasta él desde donde nos encontramos. Normalmente nuestro código de python está en un directorio concreto de nuestro ordenador, y para que lea un archivo debemos indicarle dónde se encuentra y cómo se llama (esta es su ruta). 

Podemos pedirle al lector de archivos que lea unos datos en una carpeta diferente. Para ello debemos usar la barra "/". Lo último que debe aparecer en el lector de archivos es el nombre del archivo en sí.

In [ ]:
df = pd.read_csv('data/tips.csv') # El archivo está en la carpeta "data" y se llama "tips.csv"

In [ ]:
df # Veamos qué pinta tiene...

Además del nombre del archivo con los datos, `read_csv` tiene otros argumentos opcionales. Los más utilizados son:
    
* nrows: lee solo cierto número de filas (útil para hacer una prueba antes de procesar un archivo muy grande).
* usecols: el dataframe resultante solo tendrá estas columnas
* dtype: especifica el tipo de cada columna (por defecto, pandas trata de inferirlo automáticamente de los datos, pero a veces es necesario: por ejemplo para que lea un número como string en lugar de int).

# Análisis exploratorio

Una vez ya tenemos los datos cargados, podemos echarles un vistazo mediante las siguientes cuatro funciones: `head` y `tail` para observar las primeras y últimas filas, respectivamente, `describe` para obtener algunos estadísticos básicos, e `info` para obtener datos sobre los tipos

In [ ]:
df.head(10) # Entre paréntesis, el número de entradas que queremos mostrar. Si no especificamos nada, mostrará
            # las primeras 5 filas

In [ ]:
df.tail(3) 

Existen funciones integradas que nos pueden dar una idea básica de las estadísticas de los datos, como medias, desviaciones estándar, valores mínimos y máximos y diferentes cuantiles. Esto se hace llamando a la función `describe` para datos numéricos.

In [ ]:
df.describe()

En el caso de que la tabla contenga tanto datos numéricos como categóricos, `describe` por defecto solo muestra los estadísticos para las numéricas. Si queremos que también muestre los otros:

In [ ]:
df.describe(include='all')  # Para los datos no-numéricos se obtienen "NaN" (not-a-number)

Para las variables categóricas, se muestran los nuevos estadísticos de: número de valores únicos, valor más frecuente (top) y frecuencia de éste. Por ejemplo, para el caso del sexo, hay 244 anotaciones, y el valor más frecuente es Male (157 de 244), además de tener dos valores únicos (por lo que asumiremos que el otro valor es Female)

El comando `info` nos da información extra acerca del dataframe

In [ ]:
df.info() 

## Consultas de dataframes

Hasta ahora hemos visto como acceder a ciertos registros y/o ciertas columnas de nuestra tabla. Veremos ahora algunas funciones nuevas para realizar consultas más dinámicas.

Una forma rápida de acceder a una columna entera es

In [ ]:
df['tip'].head()

También se podría hacer

In [ ]:
df.tip.head()

Gracias a que hemos aprendido numpy, como pandas va sobre numpy podemos utilizar una sintaxis muy similar

In [ ]:
df['size'].median(), df['size'].mean()   # La mediana y la media de la ocupación de las mesas de los clientes

Si queremos ver qué mesas tienen una ocupación mayor que la media, podemos hacer indexing muy parecido a cómo lo hacíamos con numpy

In [ ]:
df['size'] > df['size'].median()

Lo anterior nos ha generado un array booleano, ¡y por tanto podemos utilizarlo para seleccionar filas en nuestro dataframe!

In [ ]:
mesas_grandes = df[ df['size'] > df['size'].median() ]
mesas_grandes.head()

Observamos que conserva los índices de la tabla original. Si queremos resetearlos, basta con hacer

In [ ]:
mesas_grandes.reset_index(inplace=True, drop=True)
mesas_grandes.head()

Los criterios de filtrado de filas pueden ser tan complejos como queramos, utilizando los operadores lógicos

In [ ]:
# Mesas con tamaño mayor que la mediana y pagadas por mujeres
df[ (df['size'] > df['size'].median()) & (df['sex'] == 'Female') ].head()

## Ejercicio: Medallas olímpicas

Carga la tabla `olympics.csv`. Si leemos el archivo sin prestar mucha atención o mirarlo antes, veremos que da un ligero problema: 

In [ ]:
pd.read_csv('data/olympics.csv').head()

Si observas el archivo, hay que saltarse la primera fila (skiprows). Además, indica a pandas que la primera columna sea el index (index_col=0). Realiza un análisis exploratorio de los datos

In [ ]:
olympics = pd.read_csv('data/olympics.csv', skiprows=1, index_col=0)
olympics.head() # Comprueba la diferencia respecto al caso de antes, ¿ha mejorado en algo?

Para ayudarnos en la exploración, ejecuta el siguiente código para renombrar las columnas

In [ ]:
for col in olympics.columns:
    if col[:2]=='01':
        olympics.rename(columns={col:'Gold' + col[4:]}, inplace=True)
    if col[:2]=='02':
        olympics.rename(columns={col:'Silver' + col[4:]}, inplace=True)
    if col[:2]=='03':
        olympics.rename(columns={col:'Bronze' + col[4:]}, inplace=True)
    if col[:1]=='№':
        olympics.rename(columns={col:'#' + col[1:]}, inplace=True) 
        
names_ids = olympics.index.str.split('\s\(') # Partimos los índices en los "("

olympics.index = names_ids.str[0] # El primer elemento [0] en los índices partidos es el nombre del país 
olympics['ID'] = names_ids.str[1].str[:3] # El segundo elemento [1] es el ID (tomamos solo los siguientes 3 caracteres)
        
olympics =  olympics.drop('Totals') # Obviamos la columna de "Totals"
olympics.head()

In [ ]:
olympics.describe()

Ahora te toca a ti. Responde a la siguiente preguntas usando el dataframe procesado:
* ¿Qué país ha obtenido más medallas de oro en juegos de verano? ¿Qué índice tiene el país en la tabla (número de fila)?
* ¿Qué país o países han obtenido más medallas en los JJOO de invierno que en los de verano? (cuenta cualquier tipo de medalla por igual)

In [ ]:
''' Tu código va aquí '''

In [ ]:
''' Tu código va aquí '''

Obtén los países que hayan obtenido más medallas (cuenta cada tipo de medalla por igual) en inverno que en verano 

In [ ]:
''' Tu código va aquí '''

## Creando nuevas columnas

Añadir nuevas columnas es una operación normal cuando queremos incluir nueva información en nuestro dataframe, o bien cuando pretendemos que esta columna nueva contenga información que resulta de manipular las otras columnas que ya tenemos de una manera útil para la tarea que tengamos entre manos. 

Si queremos añadir una nueva columna a nuestra tabla, es tan sencillo como realizar una nueva asignación

In [ ]:
df['new_col'] = None   # Volvamos al dataframe del restaurante e introduzcamos una columna llamada "new_col"
df.head()

También podemos crear columnas utilizando funciones de numpy, por ejemplo, ahora la rellenaremos con valores aleatorios provenientes de una distribución normal

In [ ]:
import numpy as np
df['new_col'] = np.random.randn(len(df))  # Ojo, tenemos que dar el número correcto de elementos 
                                          # para rellenar la columna entera
df.head()

También es posible combinarlos con otras columnas de la tabla, siempre que las dimensiones lo permitan (en este caso, que tengan el mismo número de elementos)

In [ ]:
del df['new_col']
df['total_bill_rand'] = np.random.randn(len(df)) + df['total_bill'] # Sumamos los valores aleatorios a la cuenta
df.head()

Una función importante de pandas es `apply`, que permite aplicar cualquier función (de numpy, pandas, o definida por nosotros) a lo largo de varias columnas.

Por ejemplo, supongamos que queremos calcular la raíz cuadrada a las columnas total_bill, tip y total_bill_rand:

In [ ]:
df[['total_bill', 'tip', 'total_bill_rand']].apply(np.sqrt).head()

In [ ]:
df.apply(np.max) # Podemos aplicar la misma función a todo el dataframe si no especificamos columnas

Ahora veremos otro ejemplo con una función creada por nosotros:

In [ ]:
def filtra_valores_pequeños(x, y):  # x es la columna de valores
    x[x<y] = 0
    return x

Podemos usar la función anterior par hacer que las facturas menores de la media valgan cero, por ejemplo. Para ello, damos como argumento a `apply` la media de la columna de las facturas con `df['total_bill'].mean()`

In [ ]:
# Podemos usar la función anterior para hacer que aquellas facturas menores que la media valgan 0 

df[['total_bill']].apply(filtra_valores_pequeños, args=[df['total_bill'].mean()]).head()

## Ejercicio: Puntos olímpicos

Trabajaremos sobre olympics. Obtén los países que hayan obtenido más puntos en inverno que en verano. Se asignan puntos de la siguiente forma: bronce 1, plata 2, oro 3.

In [ ]:
olympics['points_summer'] = ________
olympics['points_winter'] = ________

In [ ]:
'''Tu código va aquí'''

¿Qué país tiene la menor diferencia relativa de puntos entre verano e invierno? Es decir, para cada país, calcula
$$
dif\_rel = \frac{|points\_ summer - points\_winter|}{points\_summer + points\_winter} 
$$ 
y devuelve qué país tiene el valor mínimo de esta métrica nueva.

In [ ]:
import numpy as np
olympics['dif_rel'] =  __________

In [ ]:
'''Tu código va aquí'''

## Fusionando dataframes (OPCIONAL)

Es habitual tener la información repartida en dos o más tablas. Para ello, pandas cuenta con la operación `merge`, que permite fusionar registros provenientes de diferentes tablas (similar al join de SQL). Empezamos con unos datos de ejemplo:

In [ ]:
staff_df = pd.DataFrame([{'Nombre': 'Kelly', 'Rol': 'Director de RRHH'},
                         {'Nombre': 'Sally', 'Rol': 'Profesor'},
                         {'Nombre': 'James', 'Rol': 'Secretario'}])
staff_df = staff_df.set_index('Nombre')
school_df = pd.DataFrame([{'Nombre': 'James', 'Facultad': 'Negocios'},
                           {'Nombre': 'Mike', 'Facultad': 'Derecho'},
                           {'Nombre': 'Sally', 'Facultad': 'Ingeniería'}])
school_df = school_df.set_index('Nombre')
print(staff_df.head())
print()
print(school_df.head())

`merge` recibe dos tablas, y el argumento `how`, y una columna en cada tabla que servirá para establecer la correspondencia (en este caso, tomará la columna índice de cada tabla).

Veremos todas las posibilidades de este argumento para entender las diferencias.

Con `outer` la tabla combinada presenta los registros de la tabla izquierda (`staff_df`) y de la tabla derecha (`school_df`) aunque no tengan correspondencia. Por ejemplo, vemos que James era Secretario en la tabla izquierda y pertenecía a la Facultad de Negocios, así que la tabla resultante contiene esa observación. Sin embargo, Kelly sólo figuraba en la tabla Rol, por tanto aparece que su Facultad es Nan:

In [ ]:
pd.merge(staff_df, school_df, how='outer', left_index=True, right_index=True)

In [ ]:
pd.merge(staff_df, school_df, how='inner', left_index=True, right_index=True)

`left` solo tiene en cuenta aquellas claves (en este caso el Nombre) que figuren en la tabla de la izquierda:

In [ ]:
pd.merge(staff_df, school_df, how='left', left_index=True, right_index=True)

`right` es análogo a `left`, aunque evidentemente afecta solo a la tabla de la derecha:

In [ ]:
pd.merge(staff_df, school_df, how='right', left_index=True, right_index=True)

Ahora, asignaremos nuevos índices numéricos, y realizaremos un `merge`, especificando qué columnas queremos utilizar como claves en cada tabla

In [ ]:
staff_df = staff_df.reset_index()
school_df = school_df.reset_index()
pd.merge(staff_df, school_df, how='left', left_on='Nombre', right_on='Nombre')

Una observación: si además de la clave, en cada tabla tenemos columnas que se llaman igual (Ubicación en el ejemplo siguiente), al hacer el `merge` pandas añadirá sufijos para evitar la colisión de nombres

In [ ]:
staff_df = pd.DataFrame([{'Nombre': 'Kelly', 'Rol': 'Director de RRHH', 'Ubicación': 'State Street'},
                         {'Nombre': 'Sally', 'Rol': 'Profesor', 'Ubicación': 'Washington Avenue'},
                         {'Nombre': 'James', 'Rol': 'Secretario', 'Ubicación': 'Washington Avenue'}])
school_df = pd.DataFrame([{'Nombre': 'James', 'Facultad': 'Negocios', 'Ubicación': '1024 Billiard Avenue'},
                           {'Nombre': 'Mike', 'Facultad': 'Derecho', 'Ubicación': 'Fraternity House #22'},
                           {'Nombre': 'Sally', 'Facultad': 'Ingeniería', 'Ubicación': '512 Wilson Crescent'}])
print(staff_df)
print()
print(school_df)

In [ ]:
pd.merge(staff_df, school_df, how='left', left_on='Nombre', right_on='Nombre')

También es posible utilizar como clave varias columnas:

In [ ]:
staff_df = pd.DataFrame([{'Nombre': 'Sally', 'Apellido': 'Desjardins', 'Rol': 'Director de RRHH'},
                         {'Nombre': 'Sally', 'Apellido': 'Brooks', 'Rol': 'Profesor'},
                         {'Nombre': 'James', 'Apellido': 'Wilde', 'Rol': 'Secretario'}])
school_df = pd.DataFrame([{'Nombre': 'James', 'Apellido': 'Hammond', 'Facultad': 'Negocios'},
                           {'Nombre': 'Sally', 'Apellido': 'Smith', 'Facultad': 'Derecho'},
                           {'Nombre': 'Sally', 'Apellido': 'Brooks', 'Facultad': 'Ingeniería'}])


print(staff_df)
print()
print(school_df)


In [ ]:
pd.merge(staff_df, school_df, how='inner', left_on=['Nombre','Apellido'], right_on=['Nombre','Apellido'])

## Ejercicio: Medallas _per cápita_ 

Seguimos con las Olimpiadas. Carga la tabla `population.csv`, que para cada país nos da su población. Para simplificar, nos quedaremos con las poblaciones del año 2016 (haz un filtro). Después, combina las tablas olympics y population (de forma que no queden valores missing) para que en cada registro tengamos al menos el nombre del país, el número de medallas de oro, y su población. 

¿Qué país es el que tiene mayor ratio de medallas de oro por habitante?

In [ ]:
''' Importa los datos '''

In [ ]:
''' Haz el filtro '''

In [ ]:
''' Resetea los índices '''

In [ ]:
''' Combina las tablas '''

In [ ]:
''' Obtén el ratio de medallas por habitante '''

In [ ]:
''' Exporta el país con el mejor ratio '''

## Agrupando valores en dataframes (OPCIONAL)

Una función muy importante de pandas es `groupby`, con  la que podremos realizar el siguiente proceso:

* Separar la tabla en grupos en función de algún criterio
* Aplicar alguna operación a cada grupo de forma independiente
* Combinar los resultados en una nueva tabla

Empezamos con la creación de grupos. El criterio de separación más habitual es mediante el valor de alguna variable categórica, por ejemplo el sexo, el día de la semana, etc...

Vamos a hacer algunos ejemplos sobre la tabla `tips.csv`

In [ ]:
for name, group in df.groupby(by=['sex']):
    print(name)
    print(group)

También podemos especificar varias variables para agrupar. En este caso como hay dos valores por sexo, y seis por tamaño de mesa, se crearán 12 grupos

In [ ]:
for name, group in df.groupby(by=['sex', 'size']):
    print(name)
    print(group)

Una vez, hemos realizado los grupos, podemos aplicar funciones sobre ellos. Por ejemplo, vamos a calcular la media de la factura para cada grupo

In [ ]:
for name, group in df.groupby(by=['sex', 'size']):
    print(name, group['total_bill'].mean())
    # equivalente a np.mean(group['total_bill'])

No obstante, la formulación anterior puede no resultar cómoda a la hora de crear una nueva tabla con los resultados. Por eso, pandas incorpora la función `agg`:

In [ ]:
df_bill = df.groupby(['sex']).agg({'total_bill': np.mean})
df_bill

In [ ]:
df_bill = df.groupby(['sex', 'size']).agg({'total_bill': np.mean})
df_bill

Utilizando `pivot` podemos pasar la tabla a una vista matricial para ver mejor los resultados

In [ ]:
df_bill.reset_index().pivot(index='sex', columns='size')

A `agg` podemos pasarle más argumentos si queremos calcular más columnas. Con esta otra sintaxis, agg aplicará las funciones mean y std a todo el dataframe anterior (hemos seleccionado las columnas total_bill y tip previamente)

In [ ]:
df_bill_tips = df.groupby(['sex'])['total_bill', 'tip'].agg([np.mean, np.std])

In [ ]:
df_bill_tips

## Ejercicio: Facturas caras (OPCIONAL)

(_Para resolver este problema de la forma más sencilla necesitarás las funciones de las secciones anteriores. De todas formas, puede hacerse sin necesidad de ellas y con algunas líneas extras de código_)

Para la tabla de `tips.csv`, selecciona aquellas observaciones cuyas `total_bill` sean mayores que las respectivas medias + 1 desviación estándar de su día de la semana. ¿Cuántas instancias han quedado dentro de tu dataframe de comidas caras?

_Indicación_: Si has mirado las últimas lecciones de la sesión, crea primero una tabla pequeña donde para cada día de la semana aparezca la media, y luego haz un merge de esta tabla con la original, para que cada observación tenga anotada su correspondiente media + std. Una vez hecho el merge solamente hay que aplicar indexado condicionado.

In [ ]:
''' Importa los datos de nuevo y agrupa la columna "total_bill" de acuerdo a "day" '''

In [ ]:
''' Resetea los índices de la agrupación '''

In [ ]:
''' Combina los dataframes en uno '''

df_augmented= pd.merge(____ ,  _____ , how= _____ , left_on= _____ , right_on= _____)
df_augmented.head()

In [ ]:
''' Obtén el listado de las facturas pedidas y cuenta cuántas son en total '''

# Ejemplo: Predicción de salarios esperados con reglas simples

La base de datos `income` se utiliza para predecir si el salario de una persona es mayor o menor de 50.000 dolares anuales. Para cada persona, contiene la siguiente información:
* `age`: La edad de la persona en cuestión.
* `education`: nivel de estudios.
* `marital.status`: estado civil.
* `relationship`: puesto que ocupa en su familia.
* `race`: raza.
* `sex`: género.
* `hourspeerweek`: horas de trabajo por semana.
* `nativecountry`: país de origen.
* `income`: variable que indica si el salario es mayor o menor de 50k.

Lee los datos, y elimina cualquier fila que contenga algún valor missing. Puedes utilizar en método `dropna`. También, por comodidad, sustituye los valores de `Income` por 0 (si el salario es menor o igual a 50k) y 1 (si es mayor).

In [ ]:
inc = pd.read_csv('data/income.csv')
inc.dropna(inplace=True)
inc.loc[inc.income == '<=50K', "income"] = 0
inc.loc[inc.income == '>50K', "income"] = 1
inc.head(20)

Elabora una regla para predecir la variable `income`, y almacena los resultados en la variable `income_pred`. Puedes comprobar cómo de buena es tu regla utilizando la función acierto, que recibe dos vectores, `income` que es el valor real e `income_pred` que es la predicción.

In [ ]:
# Función para evaluar el porcentaje de acierto

def acierto(income, income_pred):
    
    # income        :    valor real de los ingresos para la persona dada
    # income_pred   :    valor predicho de los ingresos para esa misma persona
    
    ac = sum(income == income_pred)*100/(len(income))
    return "Porcentaje de acierto: " + str(ac) + "%"


Este es un ejemplo muy sencillo, con lo que puedes diseñar la regla que consideres más apropiada. La que propondremos, de otra forma, es la siguiente: diremos que cobran más de 50k aquelos cuyo nivel de estudios sea master o doctorado y que además su edad esté por encima de los 30 años. Con la regla que escojas, evalúa el porcentaje de acierto. ¿Qué tal funciona tu regla?

In [ ]:
inc["income_pred"] = 0

condicion = (inc.education.isin(["Doctorate", "Masters"])) &\
(inc.age > 30) # | (inc.nativecountry == 'Cuba') # podemos hacer la regla tan compleja como queramos
inc.loc[condicion, "income_pred"] = 1
acierto(inc.income, inc.income_pred)